In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark-dataPrep") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.5") \
    .getOrCreate()

In [ ]:
spark.read.csv("s3://502finalprojbucky/congresstweets/data/June2017.csv/*.part",header=True)